<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/240726_npp_analysis_cotubanama_national_park.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

ee.Authenticate()
ee.Initialize(project='ee-foss4g')

In [7]:
# Cotubanama National Park
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
# wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 180))
wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 768))

wdpa_name = wdpa.first().get('NAME').getInfo()
print("Name:", wdpa_name)

Name: Seoraksan


In [8]:
# 1: Pixel value for the outline, 2: Thickness of the outline (in pixels)
wdpa_raster = ee.Image().paint(wdpa, 1, 2)

# Set visualization parameters
wdpa_vis = {
    'palette': ['#FFFF00'], # Apply Yellow color
    'opacity': 1
}

m = geemap.Map(width="800px", height="400px")
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [41]:
dataset = (
    ee.ImageCollection('MODIS/061/MOD17A3HGF')
    .filterDate("2023-01-01", "2023-12-31")
    .filterBounds(wdpa)
)

npp = dataset.select('Npp').first().clip(wdpa.geometry()).multiply(0.0001)

crs = ee.Projection('EPSG:3857')
npp = npp.reproject(crs=crs, scale=500)

npp_vis = {
    'min': 0,
    'max': 1.9,
    'palette': ['bbe029', '0a9501', '074b03']
}

m = geemap.Map(width="800px", height="400px",
               basemap='Esri.WorldImagery')
m.addLayer(npp, npp_vis, 'NPP')
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import time

# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=npp,
    description='NPP_2023',
    folder='export',
    scale=500,  # 이미지의 해상도
    region=wdpa.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: CJYETPU6KACFTA5F57W4X357). Status: READY
